# Inloggen en authenticatie

> tags: []

Learn to set up Spring Security to restrict access to views in a Flow application.


Met het inlogscherm op zijn plaats, moet u nu Spring Security configureren om de verificatie uit te voeren en om te voorkomen dat onbevoegde gebruikers toegang krijgen tot weergaven.

### [](https://vaadin.com/docs/latest/tutorial/login-and-authentication/#installing-spring-security-dependencies)Spring Security-afhankelijkheden installeren

Voeg de Spring Security-afhankelijkheid toe in pom.xml:

```
<dependency>
    <groupId>org.springframework.boot</groupId>
    <artifactId>spring-boot-starter-security</artifactId>
</dependency>
```

Controleer of de afhankelijkheid is gedownload. Als u het niet zeker weet, voert u vanaf de opdrachtregel uit om de afhankelijkheid te downloaden.`./mvnw install`

### [](https://vaadin.com/docs/latest/tutorial/login-and-authentication/#configuring-spring-security)Voorjaarsbeveiliging configureren

Maak een nieuw pakket voor klassen met betrekking tot beveiliging.`com.example.application.security`

<table><tbody><tr><td _msthidden="1"><p>Tip</p></td><td><p>Klassen automatisch maken</p><p><span _mstmutation="1" _msthash="3190824" _msttexthash="3964363">Plak de klassecode in het pakket zodat IntelliJ de klasse automatisch voor u maakt.</span><code>security</code></p></td></tr></tbody></table>

Schakel Spring Security in en configureer deze met een nieuwe klasse, :`SecurityConfig.java`

```
package com.example.application.security;

import com.example.application.views.LoginView;
import com.vaadin.flow.spring.security.VaadinWebSecurityConfigurerAdapter;
import org.springframework.context.annotation.Bean;
import org.springframework.context.annotation.Configuration;
import org.springframework.security.config.annotation.web.builders.HttpSecurity;
import org.springframework.security.config.annotation.web.builders.WebSecurity;
import org.springframework.security.config.annotation.web.configuration.EnableWebSecurity;
import org.springframework.security.core.userdetails.User;
import org.springframework.security.core.userdetails.UserDetailsService;
import org.springframework.security.provisioning.InMemoryUserDetailsManager;

@EnableWebSecurity 1
@Configuration
public class SecurityConfig extends VaadinWebSecurityConfigurerAdapter { 2

    @Override
    protected void configure(HttpSecurity http) throws Exception {
        super.configure(http);

        setLoginView(http, LoginView.class); 3
    }

    /**
     * Allows access to static resources, bypassing Spring security.
     */
    @Override
    public void configure(WebSecurity web) throws Exception {
        web.ignoring().antMatchers("/images/**"); 4
        super.configure(web);
    }

    /**
     * Demo UserDetailService, which only provides two hardcoded
     * in-memory users and their roles.
     * NOTE: This should not be used in real-world applications.
     */
    @Bean
    @Override
    public UserDetailsService userDetailsService() {
        return new InMemoryUserDetailsManager(User.withUsername("user") 5
            .password("{noop}userpass")
            .roles("USER")
            .build());
    }
}
```

1.  Schakel Veerbeveiliging in.
    
2.  Uitbreiden van om Spring Security voor Vaadin te configureren.`VaadinWebSecurityConfigurerAdapter`
    
3.  Sta toegang tot de inlogweergave toe.
    
4.  Sluit de afbeeldingsmap uit om openbare toegang toe te staan.
    
5.  Configureer een gebruiker in het geheugen om te testen (zie onderstaande opmerking).
    

<table><tbody><tr><td _msthidden="1"><p>Warning</p></td><td><p>Gebruik nooit hardgecodeerde referenties in de productie</p><p _msthash="3190825" _msttexthash="37626745">Gebruik geen hardgecodeerde referenties in echte toepassingen. U kunt de Configuratie van Spring Security wijzigen om een verificatieprovider te gebruiken voor LDAP, JAAS en andere bronnen uit de echte wereld. <a href="https://dzone.com/articles/spring-security-authentication" _istranslated="1">Lees meer over Verificatieproviders van Spring Security</a>.</p></td></tr></tbody></table>

Maak vervolgens in hetzelfde pakket een service voor toegang tot informatie over de aangemelde gebruiker en voor het uitloggen van de gebruiker.

```
package com.example.application.security;

import com.vaadin.flow.component.UI;
import com.vaadin.flow.server.VaadinServletRequest;
import org.springframework.security.core.context.SecurityContext;
import org.springframework.security.core.context.SecurityContextHolder;
import org.springframework.security.core.userdetails.UserDetails;
import org.springframework.security.web.authentication.logout.SecurityContextLogoutHandler;
import org.springframework.stereotype.Component;

@Component
public class SecurityService {

    private static final String LOGOUT_SUCCESS_URL = "/";

    public UserDetails getAuthenticatedUser() {
        SecurityContext context = SecurityContextHolder.getContext();
        Object principal = context.getAuthentication().getPrincipal();
        if (principal instanceof UserDetails) {
            return (UserDetails) principal;
        }
        // Anonymous or no authentication.
        return null;
    }

    public void logout() {
        UI.getCurrent().getPage().setLocation(LOGOUT_SUCCESS_URL);
        SecurityContextLogoutHandler logoutHandler = new SecurityContextLogoutHandler();
        logoutHandler.logout(
                VaadinServletRequest.getCurrent().getHttpServletRequest(), null,
                null);
    }
}
```

Voeg ten slotte annotaties toe aan beide weergaven zodat alle ingelogde gebruikers er toegang toe hebben.`@PermitAll`

```
@PermitAll
@Route(value="", layout = MainLayout.class)
@PageTitle("Contacts | Vaadin CRM")
public class ListView extends VerticalLayout {
    // omitted
}
```

```
@PermitAll
@Route(value = "dashboard", layout = MainLayout.class)
@PageTitle("Dashboard | Vaadin CRM")
public class DashboardView extends VerticalLayout {
    // omitted
}
```
